In [2]:
# deleting profile and radmass files
import os

if os.path.exists('profileSL.dat'):
    os.remove('profileSL.dat')

if os.path.exists('radmassSL.dat'):
    os.remove('radmassSL.dat')

In [3]:
# source: https://www.codesansar.com/numerical-methods/
#         runge-kutta-fourth-order-rk4-python-program.htm

import cmath #To help us out with the complex square root
import numpy as np #For the arrays
import matplotlib.pyplot as plt #Visualization

# deleting profile and radmass files
# os.remove("radmass.txt")
# os.remove("profile.txt")

# some constants
GS = 1.325 * 10**(-12) # Newton constant in fm^3 / MeV m^2
MSS = 1.1155 * 10**(15) # Sun's mass in MeV m^3 / fm^3
PI = np.pi 
HC = 197.327 # hc=1=197.327 MeV fm
KN = 8*PI*GS

# constants in eden
B = 57 # MeV
w = 1/3. # speed of sound
a4 = 0.07 # don't know, dimensionless
ms = 100 # quark mass in MeV

# additional constants for tangential pressure
BT = 57 # MeV
a4t = 0.7 # don't know, dimensionless

# constants for SL model
ctilde = 1.e0 # meter^2

# masalah satuan 
alpha = ms**2/(3*PI*np.sqrt(a4))*HC**(-1.5)
beta = ms**4/(12*PI**2)*(1-1/a4)*HC**(-3)
gamma = 8*PI/(3*ms**2*np.sqrt(a4))*HC**(1.5)
kappa = 3/(1-1/a4)

alphat = ms**2/(3*PI*np.sqrt(a4t))*HC**(-1.5)
betat = ms**4/(12*PI**2)*(1-1/a4t)*HC**(-3)
gammat = 8*PI/(3*ms**2*np.sqrt(a4t))*HC**(1.5)
kappat = 3/(1-1/a4t)

In [4]:
# define energy density as function of pressure

def eden(P):
    sqdoe = alpha**2 + 4*w*P + 12*w**2*B
    doeden = alpha**2/(2*w**2) + alpha/(2*w**2)*np.sqrt(sqdoe)
    lgPt = gamma*np.sqrt(P/w + 3*B + doeden)
    Ptilde = P - beta*(1+kappa*np.log(lgPt))
    sqde = alpha**2 + 4*w*Ptilde + 12*w**2*B
    deden = alpha**2/(2*w**2) + alpha/(2*w**2)*np.sqrt(sqde)
    # print("cek eden: ",sqdoe,doeden,lgPt,Ptilde,sqde,deden)
    return P/w + 4*B + deden

def dedP(P):
    dP = 0.00001
    x1 = P-2*dP
    x2 = P-dP
    x3 = P+dP
    x4 = P+2*dP
    dedP = eden(x1) - 8*eden(x2) + 8*eden(x3) - eden(x4)
    dedP = dedP/(12*dP)
    return dedP

# define tangential pressure

def fpt(ed,PCC):
    fpt = PCC + w*(ed-4*BT) - alphat*np.sqrt(ed-BT)
    fpt = fpt + betat*(1+kappat*np.log(gammat*np.sqrt(ed-BT)))
    fpt = fpt - w*(eden(PCC)-4*BT) + alphat*np.sqrt(eden(PCC)-BT)
    fpt = fpt - betat*(1+kappat*np.log(gammat*np.sqrt(eden(PCC)-BT)))
    return fpt

def dPTde(ed):
    return w-(1/2)*alphat/np.sqrt(ed-BT)+(1/2)*betat*kappat/(ed-BT)

# define sigma

def sig(P,PCC):
    ed = eden(P)
    Ptan = fpt(ed,PCC)
    sig = P - Ptan
    return sig

def dsdP(P,PCC):
    dP = 0.00001
    x1 = P-2*dP
    x2 = P-dP
    x3 = P+dP
    x4 = P+2*dP
    dsdP = sig(x1,PCC) - 8*sig(x2,PCC) + 8*sig(x3,PCC) - sig(x4,PCC)
    dsdP = dsdP/(12*dP)
    return dsdP


# the TOV SL equation

def b1(r,P,m,b,Lambda):
    Pcorr0 = Lambda - (1/2)*(eden(P)+P)**2 - 2*sig(P,PCC)*(eden(P)+P) 
    Pcorr = P - (KN/4)*ctilde*Pcorr0
    return (4*PI*r**3*Pcorr+MSS*m)*2*GS/(2*r*(r-2*GS*MSS*m))*2*b

def P1(r,P,m,b,Lambda):
    return -(eden(P)+P) * b1(r,P,m,b,Lambda)/(2*b) -2*sig(P,PCC)/r

def m1(r,P,m,b,Lambda):
    edencorr0 = Lambda - (3/2)*(eden(P)+P)**2 + 2*sig(P,PCC)*(eden(P)+P) 
    edencorr = eden(P) + (KN/4)*ctilde*edencorr0
    return 4*PI*r**2*edencorr/MSS

def L1(r,P,m,b,Lambda):
    ed = eden(P)
    dsde=1/dedP(P)-dPTde(ed)
    sigprime = dsdP(P,PCC)*P1(r,P,m,b,Lambda)
    sigprime = sigprime + dsde*dedP(P)*P1(r,P,m,b,Lambda)
    Lambda1 = P1(r,P,m,b,Lambda)*(eden(P)+P-2*sig(P,PCC))*(1-dedP(P))
    Lambda1 = Lambda1 + 8*sig(P,PCC)/r*(eden(P)+P-sig(P,PCC))
    Lambda1 = Lambda1 + 2*sigprime*(eden(P)+P)
    return Lambda1

In [5]:
# define the Runge-Kutta 4th order for the problem
# if we want to print the profile, set profile=1
# if we not, set profile=0
def RungeKutta(rCC,bCC,LCC,PCC,MCC,h,profile):
    # input initial values
    r0 = rCC
    b0 = bCC
    P0 = PCC
    m0 = MCC
    L0 = LCC
    while (P0 > 0.):
        if profile == 1:
            f0 = 1-2*GS*MSS*m0/r0
            print(r0/1000, b0, P0, m0, L0/1.e+6, f0, file=open('profileSL.dat', 'a'))
        # calculate k1
        r01 = r0
        b01 = b0
        P01 = P0
        m01 = m0
        L01 = L0
        k1_b = h * b1(r01,P01,m01,b01,L01)
        k1_P = h * P1(r01,P01,m01,b01,L01)
        k1_m = h * m1(r01,P01,m01,b01,L01)
        k1_L = h * L1(r01,P01,m01,b01,L01)
        # calculate k2
        r01 = r0 + h/2
        b01 = b0 + k1_b/2
        P01 = P0 + k1_P/2
        m01 = m0 + k1_m/2
        L01 = L0 + k1_L/2
        k2_b = h * b1(r01,P01,m01,b01,L01)
        k2_P = h * P1(r01,P01,m01,b01,L01)
        k2_m = h * m1(r01,P01,m01,b01,L01)
        k2_L = h * L1(r01,P01,m01,b01,L01)
        # calculate k3
        r01 = r0 + h/2
        b01 = b0 + k2_b/2
        P01 = P0 + k2_P/2
        m01 = m0 + k2_m/2
        L01 = L0 + k2_L/2
        k3_b = h * b1(r01,P01,m01,b01,L01)
        k3_P = h * P1(r01,P01,m01,b01,L01)
        k3_m = h * m1(r01,P01,m01,b01,L01)
        k3_L = h * L1(r01,P01,m01,b01,L01)
        # calculate k4
        r01 = r0 + h
        b01 = b0 + k3_b
        P01 = P0 + k3_P
        m01 = m0 + k3_m
        L01 = L0 + k3_L
        k4_b = h * b1(r01,P01,m01,b01,L01)
        k4_P = h * P1(r01,P01,m01,b01,L01)
        k4_m = h * m1(r01,P01,m01,b01,L01)
        k4_L = h * L1(r01,P01,m01,b01,L01)
        # calculate the next r0, P0, m0, and b0
        r0 = r0 + h
        b0 = b0 + (k1_b+2*k2_b+2*k3_b+k4_b)/6
        P0 = P0 + (k1_P+2*k2_P+2*k3_P+k4_P)/6
        m0 = m0 + (k1_m+2*k2_m+2*k3_m+k4_m)/6
        L0 = L0 + (k1_L+2*k2_L+2*k3_L+k4_L)/6
        f0 = 1-2*GS*MSS*m0/r0
        # print(P0)
    if profile == 1:
        f0 = 1-2*GS*MSS*m0/r0
        print(r0/1000, b0, P0, m0, L0/1.e+6, f0, file=open('profileSL.dat', 'a'))
    # the results at the surface
    output = np.array([r0,b0,P0,m0,L0,f0])
    return output
# define for a single PCC

def single_PCC(x,radmassdata,profile):
    # define initial parameters
    rCC = 1.*10**(-12) # radius near center in m--the starting point
    rmax = 100000. # radius at far distances in m
    PCC = x # pressure at the center in MeV / fm^3
    MCC = (4*PI/3)*eden(PCC)*rCC**3/MSS # Mass at the center in MeV m^3 / fm^3
    bCC = 1-2*GS*MSS*MCC/rCC # metric function b(r) at the center
    LCC = 1.*10**(-12) # Lambda function at center

    h = 10. # h-step

    # calculate the surface values
    output=RungeKutta(rCC,bCC,LCC,PCC,MCC,h,0)
    # print(output)

    # at the surface, b = 1-2Gm/r, which is different to the result
    rSurface=output[0]
    bSurface=output[1]
    mSurface=output[3]
    bSurfaceTarget = 1-2*GS*MSS*mSurface/rSurface

    # So, we redefine bCC as follows
    bCorrection=bSurfaceTarget/bSurface
    # print("cek bCC ",abs(bCorrection-1))

    # if abs(LSurface) not near 0, then recalculate
    LSurface=output[4]
    print(PCC, rSurface, mSurface, GS*MSS*mSurface/rSurface, bCorrection, LSurface)

    # if abs(bCorrection) not near 1, then recalculate
    while (abs(bCorrection-1)>10**(-3)):
        while (abs(LSurface)>10**(-1)):
            print("abs(LSurface)>10**(-1)")
            if ctilde==0:
                break
            LCC = LCC - LSurface
            output=RungeKutta(rCC,bCC,LCC,PCC,MCC,h,0)
            LSurface=output[4]
            # print("cek LCC ",(LSurface))
            # print(output)
        print("abs(bCorrection-1)>10**(-3)")
        bCC=bCC*bCorrection
        output=RungeKutta(rCC,bCC,LCC,PCC,MCC,h,0)
        rSurface=output[0]
        bSurface=output[1]
        mSurface=output[3]
        bSurfaceTarget = 1-2*GS*MSS*mSurface/rSurface
        bCorrection=bSurfaceTarget/bSurface

    rSurface=output[0]
    bSurface=output[1]
    mSurface=output[3]
    bSurfaceTarget = 1-2*GS*MSS*mSurface/rSurface
    bCorrection=bSurfaceTarget/bSurface
    LSurface=output[4]
    print(PCC, rSurface, mSurface, GS*MSS*mSurface/rSurface, bCorrection, LSurface)

    if profile == 1:
        output=RungeKutta(rCC,bCC,LCC,PCC,MCC,h,profile)
        # print(output)
    if radmassdata == 1:
        print(PCC, (eden(PCC)/1000), (rSurface/1000), mSurface, 
            2*GS*MSS*mSurface/rSurface, LCC/1.e+6, 0.5*np.log(bCC),
            file=open('radmassSL.dat', 'a'))

    output2=np.array([PCC, (eden(PCC)/1000), (rSurface/1000), mSurface, GS*MSS*mSurface/rSurface])
    return output2

In [6]:
# this is for multiple PCC
for x in range(5, 501, 5):
    PCC=x
    if PCC==500:
        single_PCC(PCC,1,1)
    else:
        single_PCC(PCC,1,0)

5 4010.000000000001 0.07302936766955574 0.02691774165009749 0.9192893383617102 5242.961060318566
abs(LSurface)>10**(-1)
abs(bCorrection-1)>10**(-3)
5 4010.000000000001 0.07302936765832904 0.02691774164595947 1.0000000000011664 1.7506230207686713e-06
10 5380.000000000001 0.18030297411999757 0.049534304295703685 0.8515197320434299 10541.940578315303
abs(LSurface)>10**(-1)
abs(bCorrection-1)>10**(-3)
10 5380.000000000001 0.1803029740629252 0.0495343042800243 1.0000000000080989 4.113393096361051e-06
15 6240.000000000001 0.28701589249600984 0.06798401477645369 0.7962452268203232 16345.321983981388
abs(LSurface)>10**(-1)
abs(bCorrection-1)>10**(-3)
15 6240.000000000001 0.28701589235313363 0.06798401474261132 1.000000000024009 9.592734912189371e-06
20 6860.000000000001 0.38877584057385545 0.08376461683122152 0.7489435330938229 22332.081285429467
abs(LSurface)>10**(-1)
abs(bCorrection-1)>10**(-3)
20 6860.000000000001 0.3887758403074011 0.08376461677381197 1.0000000000503761 1.4459643587372284e